1000 Steps Random Walk
---
Consider a 1000-state version of the random walk task. The states are numbered from `1 to 1000`, left to right, and all episodes begin near the center, in state `500`. State transitions are from the current state to one of the `100` neighboring states to its left, or to one of the `100` neighboring states to its right, all with equal probability. Of course, if the current state is near an edge, then there may be fewer than `100` neighbors on that side of it. In this case, all the probability that would have gone into those missing neighbors goes into the probability of terminating on that side (thus, state 1 has a 0.5 chance of terminating on the left, and state 950 has a 0.25 chance of terminating on the right). As usual, termination on the left produces a reward of `-1`, and termination on the right produces a reward of `+1`. All other transitions have a reward of `zero`.

In [ ]:
NUM_STATES = 1000
START = 500
END_0 = 0
END_1 = 20